In [1]:
import keepa
import numpy as np
import requests
import matplotlib.pyplot as plt

In [2]:
with open("api_key.txt") as key_file :
    api_key = key_file.readline().strip()
api = keepa.Keepa(api_key)

In [9]:
book_params = {
    'deltaPercent7_USED_lte': -50,
    'deltaPercent90_COUNT_USED_lte': 50,
    'avg180_SALES_lte': 7000000,
    'rootCategory': 283155
}
# don't include list price because there's no way to compare the list price with the used price at this point in the process
# have Mark go through the other parameters and see if there's anything he thinks would also be helpful to include

In [10]:
good_books = api.product_finder(book_params)

In [41]:
test_params = {
    'avg180_LISTPRICE_gte': 50,
    'avg180_LISTPRICE_lte': 10000000,
    'current_LISTPRICE_gte': 50,
    'current_LISTPRICE_lte': 10000000,
    'avg30_USED_gte': 12,
    #'avg30_USED_lte': 1000000,
    'current_USED_lte': 8,
    'avg180_SALES_lte': 7000000,
    'rootCategory': 283155
}
# won't give me more than 50 books, and they're all novels with used listings for just a few cents 
# and none of their list prices are over $50

In [42]:
test2 = api.product_finder(test_params)

In [44]:
params = {
    'current_LISTPRICE_gte': 50,
    'rootCategory': 283155
}

In [45]:
price = api.product_finder(params)

In [46]:
price

['B00N31ZJ0O',
 '0735219095',
 '1641526270',
 'B07FSXPMHY',
 '1984801252',
 '0735224315',
 '0062820184',
 'B07Z44CWFM',
 '0593192575',
 'B07YNZTC68',
 'B074F3BX79',
 '1338635174',
 'B0844TK94Y',
 '0062841742',
 '3869931000',
 '0374300216',
 'B07B3JQZCL',
 '014313566X',
 'B07KKP62FW',
 'B07NY9RLK1',
 'B085LQ43LJ',
 '0385348711',
 'B0731QJ482',
 '1641526378',
 'B07RFSSYBH',
 '1641522631',
 'B07Y2FCWTC',
 '1939754666',
 '1250209765',
 'B07S84VPXK',
 '0399175210',
 'B07NJCG1XS',
 '006282015X',
 'B000R34YKC',
 '0794444903',
 'B0006IU4C0',
 '1250301696',
 '0385545932',
 '0736440208',
 'B0835YQDTQ',
 'B01COR1GM2',
 '0761193642',
 '0143036491',
 '1492669733',
 '0062976583',
 'B07RQ9LR1K',
 'B008BUHTLE',
 '0545392551',
 '0399590501',
 'B07JX7ZDJ3']